In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import os.path as pth
import datetime

In [2]:
data_base_path = pth.join('data', 'public')
submission_csv_name = 'sample_submisstion.csv'

submission_base_path = pth.join(data_base_path, 'submission')
os.makedirs(submission_base_path, exist_ok=True)

In [3]:
dfs = []
sources = []
for file in glob('./data/public/submission/20201111*'):
    sources.append(file.split('_')[1].replace('-for-upload', ''))
    dfs.append(pd.read_csv(file))

merged_df = pd.concat(dfs)

In [4]:
cnt_per_id = merged_df.groupby(by=['id', 'landmark_id'])['id'].count().rename('cnt').to_frame()
cnt_per_id['cnt_max'] = cnt_per_id.groupby(by=['id', 'landmark_id'])['cnt'].max()

In [5]:
merged_df = pd.merge(merged_df, cnt_per_id, left_on=['id', 'landmark_id'], right_index=True)
#merged_df['max'] = merged_df.groupby(by=['id', 'landmark_id'])['cnt'].max()
merged_df = merged_df[merged_df['cnt'] == merged_df['cnt_max']]
merged_df = merged_df.groupby(by='id').apply(lambda x: x.nlargest(1, columns=['conf']))[['id', 'landmark_id', 'conf']]
merged_df.reset_index(drop=True, inplace=True)

In [6]:
# load template
submission_csv_path = pth.join(data_base_path, submission_csv_name)
submission_df = pd.read_csv(submission_csv_path)

del submission_df['landmark_id']
del submission_df['conf']

In [7]:
submission_df = pd.merge(submission_df, merged_df, on=['id'], how='left')

In [8]:
today_str = datetime.date.today().strftime('%Y%m%d')
result_filename = 'Merged_{}.csv'.format('_'.join(sources))

submission_csv_fileaname = pth.join(submission_base_path, '_'.join([today_str, result_filename]))
submission_df.to_csv(submission_csv_fileaname, index=False)